In [1]:
import pandas
import os
import glob
from datetime import datetime, timedelta
import dask

import plotly.offline as py
py.init_notebook_mode(connected=True)
import plotly.graph_objs as go

In [2]:
data_folder = os.path.expanduser('~/localdatalake/tse_refined')

In [3]:
def aggregate(filename):
    df = pandas.read_csv(filename, sep=';')
    flter = df['Cargo'].isin([1,3,11])
    columns_agg = [x for x in df.columns if x.startswith('Votos')]
    columns_grouby = [x for x in df.columns if (x not in columns_agg) and (x not in ['Zona','Seção'])]
    df = df[flter].groupby(columns_grouby, as_index=False).agg({x: 'sum' for x in columns_agg})
    return df
files = glob.glob(os.path.join(data_folder, 'votos', 'VotoZona_*_*.csv*'))
df = pandas.concat((aggregate(x) for x in sorted(files)), copy=False, axis=0, ignore_index=True)

In [4]:
df.info(memory_usage='deep')

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 356412 entries, 0 to 356411
Data columns (total 9 columns):
Ano             356412 non-null int64
Turno           356412 non-null int64
Eleição_nome    356412 non-null object
UF              356412 non-null object
UE              356412 non-null object
Cargo           356412 non-null int64
Município       356412 non-null int64
Urna_número     356412 non-null int64
Votos           356412 non-null int64
dtypes: int64(6), object(3)
memory usage: 90.6 MB


In [5]:
df['Eleição_nome'].value_counts().sort_index()

 ELEIÇÃO SUPLEMENTAR DE MARITUBA/PA       4
 ELEIÇÃO SUPLEMENTAR DE PRIMAVERA         4
ALTO TAQUARI-MT-JUL-2017                  3
CONQUISTA D'OESTE-MT-MAR/2017             2
ELEICAO SUPLEM. DE BOA VISTA DO GURUPI    2
                                         ..
SUPLEMENTAR SIDROLÂNDIA                   2
SUPLEMENTAR SIMÕES                        2
SUPLEMENTAR TURMALINA                     2
SÃO SEBASTIÃO DA VARGEM ALEGRE ZE 178     2
ÁGUA BOA MG - 67ª ZE                      2
Name: Eleição_nome, Length: 251, dtype: int64

In [6]:
df.head()

,Ano,Turno,Eleição_nome,UF,UE,Cargo,Município,Urna_número,Votos
0,2004,1,ELEICOES 2004,AC,1007,11,1007,13,1794
1,2004,1,ELEICOES 2004,AC,1007,11,1007,15,1194
2,2004,1,ELEICOES 2004,AC,1007,11,1007,27,939
3,2004,1,ELEICOES 2004,AC,1015,11,1015,14,1155
4,2004,1,ELEICOES 2004,AC,1015,11,1015,23,879


In [7]:
df['Tipo'] = df['Urna_número'].apply(lambda x: 'BN' if x in (95,96) else 'Pessoa')

In [8]:
plot_df = (
    df[(df['Turno']==1)&(df['Cargo'].isin([1,11]))]
    .groupby(['Ano', 'Tipo'], as_index=False)
    .agg({'Votos': 'sum'})
    .sort_values(by=['Ano', 'Tipo'])
)
plot_data = [
    go.Scatter(
        x=plot_df[plot_df['Tipo']=='Pessoa']['Ano'],
        y=plot_df[plot_df['Tipo']=='Pessoa']['Votos'],
        name='Votos válidos',
        mode='lines+markers',
    ),
    go.Scatter(
        x=plot_df[plot_df['Tipo']=='BN']['Ano'],
        y=plot_df[plot_df['Tipo']=='BN']['Votos'],
        name='Brancos e nulos',
        mode='lines+markers',
    ),
]
plot_layout = dict(
    yaxis=dict(title='Pessoas', rangemode='tozero'),
    xaxis=dict(title='Ano'),
)
figure = go.Figure(data=plot_data, layout=plot_layout)
py.iplot(figure)

In [9]:
df['Ano'].value_counts()

2014    107484
2006     93861
2010     92186
2016     16630
2004     15651
2012     15533
2008     15067
Name: Ano, dtype: int64

In [10]:
df['Urna_número'].value_counts().sort_index()

10     1277
11     8059
12    14111
13    53373
14     4309
15    23600
16    16819
17     7300
18      227
19     1040
20     8070
21    15596
22     4331
23     3314
25     7450
26      580
27    17692
28    13842
29    10367
30        1
31     1571
33     1588
35       66
36     1719
40    17006
43    18585
44     7628
45    54433
50    32393
51      273
54      989
55     3269
56     1214
65     1417
70     1768
77      419
90      716
Name: Urna_número, dtype: int64

In [11]:
df['Cargo'].value_counts()

1     171838
3     121693
11     62881
Name: Cargo, dtype: int64